In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import tensorflow as tf
import random
import numpy as np
import csv
import re
import os
import create_data
from xml.dom import minidom
import xml.etree.ElementTree as ET
from vocab import Vocabulary
from env import ConceptData
from create_data import addFile
import agents

In [ ]:
NUM_OPTIONS = 10
BATCH_SIZE = 32
VOCAB_SIZE = 99
NUM_DISTRACTORS = 9
MAX_LEN = 10

In [33]:
#batch_size = 32
hidden_size = 44
num_distractors = 7
#ist an sich vorgegeben und 597 oder so
num_categories = 58
vocab_size = 99
embed_dim = 50
max_len = 5
num_epochs = 12
sender_entropy_coeff = 0.015 #wie bei Ossenkopf
receiver_entropy_coeff = 0.0 # wie bei Ossenkopf

In [32]:

vocabulary = Vocabulary()

def addFile(name):
    file_name = name + "_structured_final.xml"
    file = minidom.parse(os.path.join(os.path.join('visa_dataset', 'UK'), file_name))
    concepts = file.getElementsByTagName('concept')

    for concept in concepts:
        vocabulary.addConcept(concept)


addFile("ANIMALS")
addFile("APPLIANCES")
addFile("ARTEFACTS")
addFile("CLOTHING")
addFile("CONTAINER")
addFile("DEVICE")
addFile("FOOD")
addFile("HOME")
addFile("INSTRUMENTS")
addFile("MATERIAL")
addFile("PLANTS")
addFile("STRUCTURES")
addFile("TOOLS")
addFile("TOYS")
addFile("VEHICLES")
addFile("WEAPONS")

for concept in vocabulary.concept_list:
    vocabulary.parseConcept(concept)

input_concepts:
[<DOM Element: concept at 0x202fc0eb670>, <DOM Element: concept at 0x202fb311a60>, <DOM Element: concept at 0x202fbfab9d0>, <DOM Element: concept at 0x202fc1448b0>, <DOM Element: concept at 0x202fbf9b670>, <DOM Element: concept at 0x202fc0eb8b0>, <DOM Element: concept at 0x202fb87b040>, <DOM Element: concept at 0x202fc08ce50>, <DOM Element: concept at 0x202fbf865e0>, <DOM Element: concept at 0x202fc0eb4c0>]
target_concept:
<DOM Element: concept at 0x202fc08ce50>


In [5]:
#Initialisiere die Agents
speakerEncoder = sender.Sender(num_options=NUM_OPTIONS, batch_size=BATCH_SIZE)
speakerEncoderOnlyTarget = sender.SenderOnlyTarget(batch_size=BATCH_SIZE)
speakerDecoder = agents.Sender_LSTM(embed_dim=embed_dim, num_cells=1, hidden_size=1, max_len=MAX_LEN)
receiverEncoder = agents.Receiver(num_distractors)
receiverDecoder = agents.Receiver_LSTM(receiverEncoder, vocab_size, embed_dim, hidden_size)
guesser = agents.AuxillaryNetwork(hidden_size)

In [7]:
#prev_hidden kommt von sender; müssen wir hier faken
#prev_hidden = tf.random.normal([32,44])

In [8]:
#guesser_output = guesser(prev_hidden)

In [19]:
#the losses
#loss um prediction mit Realität zu vergleichen
#für was brauchen wir die underscore variablen?

# TODO:
# loss anpassen!

def loss(_sender_input, _message, _receiver_input, receiver_output, labels):
    """
    receiver_output ist was von receiver_sampling zurückgegeben wird
    LABELS PRINTEN IN OSSSENKOPF NOTEBOOK
    """
    acc = (labels.t() == receiver_output).float() - (labels.t() != receiver_output).float()
    return -acc, {'acc': (acc.mean().item()+1)/2}

#loss um prediction mit Realität zu vergleichen
def auxiliary_loss(receiver_thoughts, 
                  # _message, _receiver_input, 
                   guesser_output, 
                   #_labels,
                   weight=0.2):
    mae = tf.keras.losses.MeanAbsoluteError(reduction = 'none')
    loss = mae(receiver_thoughts, guesser_output,)
    loss *= weight
    return loss, {}

In [36]:
optim_guesser = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_receiver = tf.keras.optimizers.Adam(learning_rate = 1e-2)
optim_sender = tf.keras.optimizers.Adam(learning_rate = 1e-3)

In [45]:
for i in range(num_epochs):
    # 1. data holen, labels?
    ####Fehlt hier noch
    data = ConceptData(voc=vocabulary, num_distractors=NUM_DISTRACTORS, batch_size=BATCH_SIZE)
    sender_input, targets, receiver_input = data.getInput()
    
    
    ##2. durch agents laufen lassen
    output_sender = speakerEncoder(sender_input)
    message, log_prob_s, entropy_s , prev_hidden = speakerDecoder(output_sender)
    sample, log_prob_r, entropy_r, last_hidden = receiverDecoder(message, batch_size, max_len,vocab)
    guesser_output = guesser(prev_hidden)
        
    
    # 3. Loss berechnen
    auxLoss = auxiliary_loss(last_hidden, guesser_output)
    # labels fehlen noch
    loss = loss(sample, labels)
    ##policy_length_loss und das baseline Zeug miteinbringen?
    weighted_entropy = entropy_r * receiver_entropy_coeff + sender_entropy_coeff # *entropy_r
    log_prob = log_prob_r # + log_prob_s
    loss = tf.math.reduce_mean(loss* log_prob) - weighted_entropy
    
    #optimization
    
    with tf.GradientTape() as tape:
        # muss der eine Receiver im Konstruktor vom anderem erstellt werden damit das funktioniert?
        receiver_gradients = tape.gradient(loss, receiverDecoder.trainable_variables)
        print(receiver_gradients)
    optim_receiver.apply_gradients(zip(receiver_gradients, receiverDecoder.trainable_variables))
    
    with tf.GradientTape() as tape:
        guesser_gradients = tape.gradient(auxLoss, guesser.trainable_variables)
        print(guesser_gradients)
    optim_guesser.apply_gradients(zip(guesser_gradients, guesser.trainable_variables))
    
    with tf.GradientTape() as tape:
        senderLSTM_gradients = tape.gradient(loss, speakerDecoder.trainable)
        senderEncoder_gradients = guesser_gradients + senderLSTM_gradients
    optim_sender.apply_gradients(zip(senderLSTM_gradients, speakerDecoder.trainable_variables))
    optim_sender.apply_gradients(zip(senderEncoder_gradients, senderEncoder.trainable_variables))
        
    
    
    
    #testing <-> trainifehlt noch
    